In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [5]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [6]:
train_df.head()

,id,muggy-smalt-axolotl-pembus,dorky-peach-sheepdog-ordinal,slimy-seashell-cassowary-goose,snazzy-harlequin-chicken-distraction,frumpy-smalt-mau-ordinal,stealthy-beige-pinscher-golden,chummy-cream-tarantula-entropy,hazy-emerald-cuttlefish-unsorted,nerdy-indigo-wolfhound-sorted,leaky-amaranth-lizard-sorted,ugly-tangerine-chihuahua-important,shaggy-silver-indri-fimbus,flaky-chocolate-beetle-grandmaster,squirrely-harlequin-sheep-sumble,freaky-tan-angelfish-noise,lousy-plum-penguin-sumble,bluesy-rose-wallaby-discard,baggy-copper-oriole-dummy,stealthy-scarlet-hound-fepid,greasy-cinnamon-bonobo-contributor,cranky-cardinal-dogfish-ordinal,snippy-auburn-vole-learn,greasy-sepia-coral-dataset,flabby-tangerine-fowl-entropy,lousy-smalt-pinscher-dummy,bluesy-brass-chihuahua-distraction,goopy-eggplant-indri-entropy,homey-sepia-bombay-sorted,homely-ruby-bulldog-entropy,hasty-blue-sheep-contributor,blurry-wisteria-oyster-master,snoopy-auburn-dogfish-expert,stinky-maroon-blue-kernel,bumpy-amaranth-armadillo-important,slaphappy-peach-oyster-master,dorky-tomato-ragdoll-dataset,messy-mauve-wolverine-ordinal,geeky-pumpkin-moorhen-important,crabby-teal-otter-unsorted,...,beady-mauve-frog-distraction,surly-brass-maltese-ordinal,beady-asparagus-opossum-expert,beady-rust-impala-dummy,droopy-amethyst-dachshund-hint,homey-crimson-budgerigar-grandmaster,droopy-cardinal-impala-important,woozy-apricot-moose-hint,paltry-sapphire-labradoodle-dummy,crappy-carmine-eagle-entropy,greasy-magnolia-spider-grandmaster,crabby-carmine-flounder-sorted,skimpy-copper-fowl-grandmaster,hasty-seashell-woodpecker-hint,snappy-purple-bobcat-important,thirsty-carmine-corgi-ordinal,homely-auburn-reindeer-unsorted,crappy-beige-tiger-fepid,cranky-auburn-swan-novice,chewy-bistre-buzzard-expert,skinny-cyan-macaque-pembus,slimy-periwinkle-otter-expert,snazzy-burgundy-clam-novice,cozy-ochre-gorilla-gaussian,homey-sangria-wolfhound-dummy,snazzy-asparagus-hippopotamus-contributor,paltry-red-hamster-sorted,zippy-dandelion-insect-golden,baggy-coral-bandicoot-unsorted,goopy-lavender-wolverine-fimbus,wheezy-myrtle-mandrill-entropy,wiggy-lilac-lemming-sorted,gloppy-cerise-snail-contributor,woozy-silver-havanese-gaussian,jumpy-thistle-discus-sorted,muggy-turquoise-donkey-important,blurry-buff-hyena-entropy,bluesy-chocolate-kudu-fepid,gamy-white-monster-expert,target
0,707b395ecdcbb4dc2eabea00e4d1b179,-2.070654,1.018160,0.228643,0.857221,0.052271,0.230303,-6.385090,0.439369,-0.721946,-0.227027,0.575964,1.541908,1.745286,-0.624271,3.600958,1.176489,-0.182776,-0.228391,1.682263,-0.833236,-4.377688,-5.372410,-0.477742,-0.179005,-0.516475,0.127391,-0.857591,-0.461500,2.160303,-2.118371,0.515493,-1.201493,-0.027377,-1.154024,0.753204,-0.179651,-0.807341,-1.663626,0.893806,...,-1.829848,2.347131,0.082462,-1.012654,0.593752,2.904654,-0.428974,-0.919979,2.849575,-0.906744,0.729459,0.386140,0.319814,-0.407682,-0.170667,-1.242919,-1.719046,-0.132395,-0.368991,-5.112553,-2.085988,-0.897257,1.080671,-0.273262,0.342824,0.640177,-0.415298,-0.483126,-0.080799,2.416224,0.351895,0.618824,-1.542423,0.598175,0.611757,0.678772,0.247059,-0.806677,-0.193649,0
1,5880c03c6582a7b42248668e56b4bdec,-0.491702,0.082645,-0.011193,1.071266,-0.346347,-0.082209,0.110579,-0.382374,-0.229620,0.783980,-1.280579,-1.003480,-7.753201,-1.320547,0.919078,-1.036068,0.030213,0.910172,-0.905345,0.646641,-0.465291,-0.531735,-0.756781,0.193724,0.224277,-0.474412,1.885805,0.205439,-6.481422,1.035620,-0.453623,0.375936,-0.320670,-0.144646,-0.220129,0.577826,-0.360512,-0.600107,0.008111,...,0.982205,-1.161978,0.532269,1.133215,0.003503,-1.390962,0.158572,0.143794,-0.317185,1.017192,-0.395342,-0.642357,-0.627209,0.257271,-1.461564,0.325613,1.628369,0.640040,0.750735,1.164573,0.900373,0.063489,0.948158,0.273014,-1.269147,-0.251101,-2.271731,-0.044167,-0.443766,-1.144794,-0.645115,-1.246090,2.613357,-0.479664,1.581289,0.931258,0.151937,-0.766595,0.474351,0
2,4ccbcb3d13e5072ff1d9c61afe2c4f77,-1.680473,0.860529,-1.076195,0.740124,3.678445,0.28855

In [8]:
import lightgbm as lgb
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [31]:
feature = train_df.columns[1:-1]
target = train_df.columns[-1:]

In [37]:
X = train_df[feature]
y = train_df[target]
X_test = test_df[feature]

In [39]:
all_K_fold_results = []
kf = StratifiedKFold(n_splits=5, shuffle = True)
oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(test_df.shape[0])
params = {
    'metric' : 'auc'
}

for fold_, (trn_index, val_index) in enumerate(kf.split(X,y)):
    X_trn, X_val = X.iloc[trn_index], X.iloc[val_index]
    y_trn, y_val = y.iloc[trn_index], y.iloc[val_index]
    
    print("Fold:{}".format(fold_ + 1))
    
    trn_data = lgb.Dataset(X_trn, label = y_trn)
    val_data = lgb.Dataset(X_val, label = y_val)
    
    clf = lgb.train(params, trn_data, 500, valid_sets = [trn_data, val_data]
                   , verbose_eval = 100, early_stopping_rounds = 50)
    
    oof_preds[val_index] = clf.predict(X_val, num_iteration = clf.best_iteration)
    
    sub_preds += clf.predict(X_test, num_iteration = clf.best_iteration) / kf.n_splits
    
print("CV score : {:<8.5f}".format(roc_auc_score(y, oof_preds)))

Fold:1
Training until validation scores don't improve for 50 rounds.
[100]	training's auc: 0.726202	valid_1's auc: 0.535052
[200]	training's auc: 0.805216	valid_1's auc: 0.542559
[300]	training's auc: 0.85445	valid_1's auc: 0.548304
[400]	training's auc: 0.889915	valid_1's auc: 0.55129
Early stopping, best iteration is:
[446]	training's auc: 0.902573	valid_1's auc: 0.553235
Fold:2
Training until validation scores don't improve for 50 rounds.
[100]	training's auc: 0.725651	valid_1's auc: 0.53695
[200]	training's auc: 0.803677	valid_1's auc: 0.543375
[300]	training's auc: 0.855384	valid_1's auc: 0.54852
[400]	training's auc: 0.888839	valid_1's auc: 0.550469
Early stopping, best iteration is:
[387]	training's auc: 0.884671	valid_1's auc: 0.551133
Fold:3
Training until validation scores don't improve for 50 rounds.
[100]	training's auc: 0.72891	valid_1's auc: 0.540052
[200]	training's auc: 0.808251	valid_1's auc: 0.54903
[300]	training's auc: 0.857121	valid_1's auc: 0.55391
[400]	training'

In [40]:
sub_preds

array([0.51047389, 0.48745284, 0.52304279, ..., 0.47881549, 0.55564946,
       0.49167044])

In [43]:
submission = pd.read_csv("../input/sample_submission.csv")
submission[target] = sub_preds

TypeError: parser_f() got an unexpected keyword argument 'index'

In [42]:
submission.to_csv('basic_start.csv', index=False)

,id,target
0,1c13f2701648e0b0d46d8a2a5a131a53,0.510474
1,ba88c155ba898fc8b5099893036ef205,0.487453
2,7cbab5cea99169139e7e6d8ff74ebb77,0.523043
3,ca820ad57809f62eb7b4d13f5d4371a0,0.465067
4,7baaf361537fbd8a1aaa2c97a6d4ccc7,0.500757
5,8b3116e5e3e92e971dac305d1a093bf6,0.509307
6,35cfd7cab9bfa29bc963d1b8c94dd280,0.496450
7,83cf532df8ff4642a3e33a70fffdac37,0.482815
8,2e1350fe94ec9f2220bec5245e5e9265,0.505199
9,e62020afa72eb54a15725473e3a8475b,0.504053
